<a href="https://colab.research.google.com/github/gmrammohan15/EVA4/blob/master/S9-Assignment/QuizDNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
datetime.datetime.now()

datetime.datetime(2020, 3, 21, 20, 3, 13, 578208)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/gdrive/My Drive/S9-Assignment/')
!ls /content/gdrive/My\ Drive/S9-Assignment

data		    __init__.py       resnet.py
dataloader.py	    misclassified.py  S8_Assignment.ipynb
data_transforms.py  Models	      Transforms_Albumentations
eval.py		    __pycache__
grad_cam.py	    README.md


In [0]:

import numpy as np
import data_transforms
from torchvision import transforms

import importlib
importlib.reload(data_transforms)

transformer = data_transforms.Transforms(normalize=True, mean=(0.4914, 0.4822, 0.4465), stdev=(0.247, 0.243, 0.261))

#Train transformations
train_transforms = transformer.train_transforms([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
])

# Test  transformations
test_transforms = transformer.test_transforms()

In [5]:
from torchvision import datasets
train_data = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
import dataloader

import importlib
importlib.reload(dataloader)

dl = dataloader.DataLoader(batch_size=128)

# train dataloader
train_loader = dl.load(train_data)

# test dataloader
test_loader = dl.load(test_data)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class DNNNet(nn.Module):
    def __init__(self):
        super(DNNNet, self).__init__()

        self.convInput = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(1, 1), padding=0, bias=False),
        ) # output_size = 32, RF 3

        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.1),
        ) 

        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.1),
        ) 

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) 

        self.convblockx4 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1, 1), bias=False),
         ) 
    
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1),
        ) 

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.1),
        )

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        
        self.convblockx8 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1, 1), bias=False),
        ) 
        
        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
        ) 

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
        ) 

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=8)
        ) # output_size = 1
        self.fc = nn.Linear(128, 10)
        

    def forward(self, x):

        x1 = self.convInput(x) 
        x2 = self.convblock1(x) 
        x = x1 + x2

        x3 = self.convblock2(x) 

        x = x + x3

        x = self.pool1(x) 


        x5 = self.convblockx4(x) 
        x6 = self.convblock4(x5) 
        x = x5 + x6

        x7 = self.convblock5(x) 

        x = x + x7

        x = self.pool2(x) 

        x9 = self.convblockx8(x)
        x10 = self.convblock6(x9)

        x = x9 + x10

        x11 = self.convblock7(x)

        x = self.gap(x11)
        x = x.view(-1, 128)
        x = self.fc(x)

        return x

In [9]:

import torch
!pip install torchsummary
from torchsummary import summary

model = DNNNet()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = DNNNet().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]              96
            Conv2d-2           [-1, 32, 32, 32]             864
       BatchNorm2d-3           [-1, 32, 32, 32]              64
              ReLU-4           [-1, 32, 32, 32]               0
           Dropout-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 32, 32, 32]           9,216
       BatchNorm2d-7           [-1, 32, 32, 32]              64
              ReLU-8           [-1, 32, 32, 32]               0
           Dropout-9           [-1, 32, 32, 32]               0
        MaxPool2d-10           [-1, 32, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]           2,048
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

In [0]:
import torch
from tqdm import tqdm
from torch.autograd import Variable


class Train:
  def __init__(self, model, dataloader, optimizer, criterion, schedular=None, l1_lambda=0):
    self.model = model
    self.dataloader = dataloader
    self.optimizer = optimizer
    self.schedular = schedular
    self.l1_lambda = l1_lambda
    self.criterion = criterion


  def run(self, epoch):
    self.model.train()
    train_acc = 0.0
    train_loss = 0.0
    pbar = tqdm(self.dataloader)
    for i, (images, labels) in enumerate(pbar):
        images, labels = images.to(self.model.device), labels.to(self.model.device)
        correct = 0
        processed = 0
        # Clear all accumulated gradients
        self.optimizer.zero_grad()
        # Predict classes using images from the test set
        outputs = self.model(images)
        # Compute the loss based on the predictions and actual labels
        loss = self.criterion(outputs, labels)

        if self.l1_lambda > 0:
            reg_loss = 0.0
            for param in self.model.parameters():
                reg_loss += torch.sum(param.abs())
            loss += self.l1_lambda * reg_loss
        # Backpropagate the loss
        loss.backward()

        # Adjust parameters according to the computed gradients
        self.optimizer.step()
	
        y_pred = self.model(images)
        prediction = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += prediction.eq(labels.view_as(prediction)).sum().item()
        processed += len(images)

        pbar.set_description(desc= f'Epoch= {epoch} Loss={loss.item()} Batch_id={i} Accuracy={100*correct/processed:0.2f}')
        pbar.update(1)


    train_acc = 100*correct/processed

    return train_acc

class Test:
  def __init__(self, model, dataloader):
    self.model = model
    self.dataloader = dataloader

  def run(self, epoch):
    self.model.eval()
    test_acc = 0.0
    correct = 0
    total  = 0
    with torch.no_grad():
        test_pbar = tqdm(self.dataloader)
        for i, (images, labels) in enumerate(test_pbar):
            images, labels = images.to(self.model.device), labels.to(self.model.device)

            # Predict classes using images from the test set
            outputs = self.model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        #Compute the average acc and loss over all 10000 test images
    test_acc = (100 * correct / total) 

    return test_acc


class ModelTrainer:
  def __init__(self, model, optimizer, train_loader, test_loader, criterion, schedular=None, batch_schedular=False, l1_lambda = 0):
    self.model = model
    self.schedular = schedular
    self.batch_schedular = batch_schedular
    self.optimizer = optimizer
    self.criterion = criterion
    self.train = Train(model, train_loader, optimizer, self.criterion, self.schedular if self.schedular and self.batch_schedular else None, l1_lambda)
    self.test = Test(model, test_loader)

  def run(self, epochs=10):
    for epoch in range(epochs):
        train_acc = self.train.run(epoch)
        test_acc = self.test.run(epoch)
        if self.schedular and not self.batch_schedular:
            self.schedular.step()
        print("Epoch {}, Train Accuracy: {} , Test Accuracy: {}".format(epoch, train_acc, test_acc))


In [11]:
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.lr_scheduler import StepLR


EPOCHS = 30
l2_loss = 0.0001
l1_lambda = 0
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=l2_loss)

schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)
use_cuda = torch.cuda.is_available()
model.device = torch.device("cuda" if use_cuda else "cpu")
trainer = ModelTrainer(model, optimizer, train_loader, test_loader, criterion, schedular, True, l1_lambda)
trainer.run(EPOCHS)


Epoch= 0 Loss=1.2927777767181396 Batch_id=390 Accuracy=50.00: 100%|██████████| 391/391 [00:15<00:00, 24.69it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 0, Train Accuracy: 50.0 , Test Accuracy: 44.2


Epoch= 1 Loss=1.1719348430633545 Batch_id=390 Accuracy=58.75: 100%|██████████| 391/391 [00:15<00:00, 24.66it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1, Train Accuracy: 58.75 , Test Accuracy: 59.06


Epoch= 2 Loss=0.9754169583320618 Batch_id=390 Accuracy=66.25: 100%|██████████| 391/391 [00:15<00:00, 24.71it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...

Epoch 27, Train Accuracy: 93.75 , Test Accuracy: 82.74


Epoch= 28 Loss=0.5007582902908325 Batch_id=390 Accuracy=85.00: 100%|██████████| 391/391 [00:15<00:00, 24.68it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

Epoch 28, Train Accuracy: 85.0 , Test Accuracy: 84.79


Epoch= 29 Loss=0.3630760610103607 Batch_id=390 Accuracy=90.00: 100%|██████████| 391/391 [00:15<00:00, 25.08it/s]
100%|██████████| 79/79 [00:01<00:00, 42.00it/s]

Epoch 29, Train Accuracy: 90.0 , Test Accuracy: 82.96
